In [1]:
!pip install torch transformers unsloth datasets scikit-learn matplotlib seaborn gradio
import torch
if torch.cuda.is_available():
    print(f"CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU: Tesla T4


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
cache_dir_path = "/content/hf-cache"
os.environ["HF_HOME"] = "/content/hf-cache"
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import load_from_disk

In [4]:
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [7]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load DistilBERT
db_model_ckpt = "/content/drive/MyDrive/final_major_project/distibert-base-uncased-distilbert-1024/checkpoint-1280"
db_tokenizer = AutoTokenizer.from_pretrained(db_model_ckpt)
num_labels = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
db_model = AutoModelForSequenceClassification.from_pretrained(db_model_ckpt, num_labels=num_labels).to(device)

# Load CodeBERT
cb_model_ckpt = "/content/drive/MyDrive/final_major_project/codebert-base-coderbert-1024/checkpoint-1280"
cb_tokenizer = AutoTokenizer.from_pretrained(cb_model_ckpt)
cb_model = AutoModelForSequenceClassification.from_pretrained(cb_model_ckpt, num_labels=num_labels).to(device)

# Remove label from here
def create_classification_input(oldf, patch):
    code_changes = f"""
    Existing code:\n\n

    ```\n
    {oldf}
    ```

    Proposed change:\n\n

    ```\n
    {patch}
    ```
    """
    return {"code": code_changes}

def unified_inference(oldf, patch, model_choice):
    updated_entry = create_classification_input(oldf, patch)

    if model_choice == "DistilBERT":
        tokenizer = db_tokenizer
        model = db_model
    else:
        tokenizer = cb_tokenizer
        model = cb_model

    with torch.no_grad():
        inputs = tokenizer(updated_entry['code'], return_tensors='pt', truncation=True, padding=True).to(device)
        outputs = model(**inputs)
        logits = outputs.logits
        pred = torch.argmax(logits, dim=1).item()

    return "Prediction: " + str(pred)

# Gradio Interface
iface = gr.Interface(
    fn=unified_inference,
    inputs=[
        gr.Textbox(lines=10, label="Existing Code"),
        gr.Textbox(lines=5, label="Proposed Change"),
        gr.Radio(choices=["DistilBERT", "CodeBERT"], label="Select Model")
    ],
    outputs="text",
    title="Code Inference with DistilBERT and CodeBERT",
    description="Enter the existing code and proposed change, then select the model to run inference."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://646df6ad300c49c55d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
